<a href="https://colab.research.google.com/github/lstene/charlesLecrerc-performance/blob/main/F1_dataExtractions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Charles Lecrerc data extraction from 2018-2023

In [ ]:
# fastF1 is the API used for obtaining data from all the races in F1
!pip install fastf1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.3/148.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 4.4 MB/s eta 0:00:00


This is the code for getting the information of the grid position over qualy, the final position after race, points, date an

In [ ]:
import fastf1
import pandas as pd
import time

season = 2023  # Specify the season
driver_code = 'LEC'

# List to store the results for each race in the season
results = []

# Load the event schedule for the specified season
try:
    schedule = fastf1.get_event_schedule(season)

    for round_number in schedule['RoundNumber']:
        try:
            # Get the race data
            race = fastf1.get_session(season, round_number, 'R')
            race.load()  # Load the data

            # Get the race results and filter by the driver
            race_results = race.results
            leclerc_results = race_results[race_results['Abbreviation'] == driver_code]

            # Add the results to the list if data exists
            if not leclerc_results.empty:
                leclerc_results.loc[:, 'RoundNumber'] = round_number
                leclerc_results.loc[:, 'RaceName'] = race.event['EventName']
                leclerc_results.loc[:, 'EventDate'] = race.event['EventDate']

                # Tire management information: Number of pit stops and tire strategy
                driver_laps = race.laps.pick_driver(driver_code)
                pit_stops = driver_laps[driver_laps['PitInTime'].notna()]
                leclerc_results.loc[:, 'NumPitStops'] = len(pit_stops)

                # Assuming the tire compound data is available
                tire_stints = driver_laps.groupby('Stint')['Compound'].first().tolist()
                leclerc_results.loc[:, 'TireStrategy'] = ' > '.join(tire_stints)

                # Check if Leclerc had the fastest lap
                leclerc_results.loc[:, 'FastestLap'] = race.laps.pick_fastest()['Driver'] == driver_code

                # Store the result in the list
                results.append(leclerc_results)

            # Pause between requests to avoid overwhelming the server
            time.sleep(1)

        except Exception as e:
            print(f"Error processing season {season}, Round {round_number}: {e}")
            time.sleep(1)  # Wait before retrying

except ValueError as e:
    print(f"Failed to load schedule for season {season}: {e}")

# Convert the list of results into a pandas DataFrame
if results:
    df_results = pd.concat(results, ignore_index=True)
else:
    print("No data available.")


req         WARNING 	DEFAULT CACHE ENABLED! (24.0 KB) /root/.cache/fastf1


Error processing season 2023, Round 0: Cannot get testing event by round number!


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.4.0]
INFO:fastf1.fastf1.core:Loading data for Bahrain Grand Prix - Race [v3.4.0]
req            INFO 	No cached data found for session_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
INFO:fastf1.api:Fetching session info data...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_status_d

In [ ]:
print(df_results[['RoundNumber', 'RaceName', 'Position', 'GridPosition', 'Points', 'EventDate',
                       'NumPitStops', 'TireStrategy', 'FastestLap']])

    RoundNumber                  RaceName  Position  GridPosition  Points  \
0             1        Bahrain Grand Prix      19.0           3.0     0.0   
1             2  Saudi Arabian Grand Prix       7.0          12.0     6.0   
2             3     Australian Grand Prix      20.0           7.0     0.0   
3             4     Azerbaijan Grand Prix       3.0           1.0    15.0   
4             5          Miami Grand Prix       7.0           7.0     6.0   
5             6         Monaco Grand Prix       6.0           6.0     8.0   
6             7        Spanish Grand Prix      11.0          19.0     0.0   
7             8       Canadian Grand Prix       4.0          10.0    12.0   
8             9       Austrian Grand Prix       2.0           2.0    18.0   
9            10        British Grand Prix       9.0           4.0     2.0   
10           11      Hungarian Grand Prix       7.0           6.0     6.0   
11           12        Belgian Grand Prix       3.0           1.0    15.0   

For specific races where the code before extracted incorrectly the data we are going to use this query

In [ ]:
import fastf1
import pandas as pd
import time  # Necesario para la pausa entre solicitudes

# Define the specific season, driver, and round number
season = 2022  # Specify the season
driver_code = 'LEC'
round_number = 22  # Specify the race round number you're interested in

# List to store the results for the specific race
results = []

try:
    # Get the race data
    race = fastf1.get_session(season, round_number, 'R')
    race.load()  # Load the data

    # Get the race results and filter by the driver
    race_results = race.results
    leclerc_results = race_results[race_results['Abbreviation'] == driver_code]

    # Add the results to the list if data exists
    if not leclerc_results.empty:
        leclerc_results.loc[:, 'RoundNumber'] = round_number
        leclerc_results.loc[:, 'RaceName'] = race.event['EventName']
        leclerc_results.loc[:, 'EventDate'] = race.event['EventDate']

        # Tire management information: Number of pit stops and tire strategy
        driver_laps = race.laps.pick_driver(driver_code)
        pit_stops = driver_laps[driver_laps['PitInTime'].notna()]
        leclerc_results.loc[:, 'NumPitStops'] = len(pit_stops)

        # Assuming the tire compound data is available
        tire_stints = driver_laps.groupby('Stint')['Compound'].first().tolist()
        leclerc_results.loc[:, 'TireStrategy'] = ' > '.join(tire_stints)

        # Check if Leclerc had the fastest lap
        leclerc_results.loc[:, 'FastestLap'] = race.laps.pick_fastest()['Driver'] == driver_code

        # Store the result in the list
        results.append(leclerc_results)

        # Pause between requests to avoid overwhelming the server
        time.sleep(1)

except Exception as e:
    print(f"Error processing season {season}, Round {round_number}: {e}")
    time.sleep(1)  # Wait before retrying

# Convert the list of results into a pandas DataFrame
if results:
    df_results = pd.concat(results, ignore_index=True)
    print(df_results)
else:
    print("No data available.")


core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.4.0]
INFO:fastf1.fastf1.core:Loading data for Abu Dhabi Grand Prix - Race [v3.4.0]
req            INFO 	No cached data found for session_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
INFO:fastf1.api:Fetching session info data...
DEBUG:fastf1.api:Falling back to livetiming mirror (https://livetiming-mirror.fastf1.dev)
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
DEBUG:fastf1.api:Falling back to livetiming mirror (https://livetiming-mirror.fastf1.dev)
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1

  DriverNumber BroadcastName Abbreviation DriverId TeamName TeamColor TeamId  \
0           16     C LECLERC          LEC           Ferrari    ed1c24          

  FirstName LastName         FullName  ...  Q3 Time  Status Points  \
0   Charles  Leclerc  Charles Leclerc  ... NaT  NaT            NaN   

   RoundNumber              RaceName  EventDate NumPitStops   TireStrategy  \
0           22  Abu Dhabi Grand Prix 2022-11-20           1  MEDIUM > HARD   

  FastestLap  
0      False  

[1 rows x 27 columns]


In [ ]:
df_results.to_csv('season_2022.csv', index=False)